1 - Import Relevant Libraries

In [1]:
import numpy as np
from sklearn import preprocessing
# sklearn preprocessing library, will be used to standardize the data


2 - Load Data

In [2]:
raw_csv_data = np.genfromtxt('audiobooks_data.csv', delimiter=',')
raw_csv_data = raw_csv_data[2:, :]
unscaled_inputs_all = raw_csv_data[:, 1:-1]
unscaled_targets_all = raw_csv_data[:, -1]


3 - Balance Dataset b/w target = 0 and 1

In [3]:
num_one_targets = int(np.sum(unscaled_targets_all))
zero_targets_counter = 0
indices_to_remove = []
for i in range(unscaled_targets_all.shape[0]):
    if unscaled_targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)


In [4]:
unscaled_inputs_equal_priors = np.delete(
    unscaled_inputs_all, indices_to_remove, axis=0)
unscaled_targets_equal_priors = np.delete(
    unscaled_targets_all, indices_to_remove, axis=0)


4 - Standardization/Feature Scaling

In [5]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)
scaled_targets = unscaled_targets_equal_priors
# No need to scale the targets - just for symmetry and avoid confusion


5 - Shuffle Dataset

In [6]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = scaled_targets[shuffled_indices]


6 - Split Dataset (Train, Validate and Test)

In [7]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Divide the dataset into train, validation and test for inputs and targets
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count,
        np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count,
        np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count,
        np.sum(test_targets) / test_samples_count)


1789.0 3579 0.49986029617211514
226.0 447 0.5055928411633109
222.0 448 0.4955357142857143


7 - Save NPZ

In [8]:
np.savez('audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('audiobooks_data_test', inputs=test_inputs, targets=test_targets)